In [148]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [167]:
img = cv2.imread('./images/simple_binary.png')

In [168]:
X = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
X = X[0:234, 0:234]

In [169]:
p_size = 80
patches_x = 3
patches_y = 3

In [170]:
X.shape

(234, 234)

In [175]:
offset = 0
X1 = np.zeros([X.shape[0], X.shape[1], 9]) + 255
X2 = np.zeros([X.shape[0], X.shape[1], 9]) + 255
X1_U_X2 = np.zeros([X.shape[0], X.shape[1], 9])

for i in range(0, X.shape[0], p_size):
    for j in range(0, X.shape[1], p_size):
        # print(f"px = {i+1}:{i+p_size}, py = {j+1}:{j+p_size}")
        X1[i+1:i+p_size-1, j+1:j+p_size-1, offset] = np.array(X[i+1:i+p_size-1, j+1:j+p_size-1])
        
        X2[:, :, offset] = X
        X2[i:i+p_size, j:j+p_size, offset] = 255
        
        X1_U_X2[:, :, offset] = X
        X1_U_X2[i:i+p_size, j:j+p_size+1, offset] = 255 
        X1_U_X2[i+1:i+p_size-1, j+1:j+p_size-1, offset] = X1[i+1:i+p_size-1, j+1:j+p_size-1, offset] 

        cv2.imwrite(f"./images/croped/image_X1_{offset}.png", X1[:,:,offset])
        cv2.imwrite(f"./images/croped/image_X2_{offset}.png", X2[:,:,offset])
        cv2.imwrite(f"./images/croped/image_X1UX2_{offset}.png", X1_U_X2[:,:,offset])
        offset += 1

px = 1:80, py = 1:80
px = 1:80, py = 81:160
px = 1:80, py = 161:240
px = 81:160, py = 1:80
px = 81:160, py = 81:160
px = 81:160, py = 161:240
px = 161:240, py = 1:80
px = 161:240, py = 81:160
px = 161:240, py = 161:240
